***
created by **Farah Rabie** (f.rabie@hw.ac.uk)
***

#### Introduction

In this notebook, we explore how a neural network can predict **permeability** (KLOGH) from well log measurements. We look into:

*   **Preprocess** the well log data, focusing on sandstone intervals.
*   **Examine** relationships between input features and the target using Spearman’s correlation.
*   **Train** a feedforward neural network on processed data.
*   **Evaluate** the trained network to see how well it captures the trends.

##### I$\,\,\,\,\,\,$Data Visualisation

First, let's visualise the data. To do so, we import the `VisualiseWellData` class and create a visualiser instance.

In [ ]:
from Lib.DataVisualisation import VisualiseWellData
visualiser = VisualiseWellData()

Then, we list the file paths and names of all available well datasets to be used in this notebook.

In [ ]:
# List of well file paths
well_files = [
    r".\Well Data\15_9-F-4.csv",
    r".\Well Data\15_9-F-5.csv",
    r".\Well Data\15_9-F-11 B.csv",
    r".\Well Data\15_9-F-12.csv",
    r".\Well Data\15_9-F-14.csv",
    r".\Well Data\15_9-F-15 C.csv"
]

# List of corresponding well names
well_names = [
    "15_9-F-1 B",
    "15_9-F-4",
    "15_9-F-5",
    "15_9-F-11 B",
    "15_9-F-12",
    "15_9-F-14",
    "15_9-F-15 C"
]

##### II$\,\,\,\,\,\,$Data Processing

Next, we import the DataProcessing class from the `NeuralNetworkFunctions` module and create an instance called `DataProcess`. Similar to earlier exercises, this object will be used to handle the processing of the well log data.

In [ ]:
from Lib.NeuralNetworkFunctions import DataProcessing
DataProcess = DataProcessing()

We define separate lists for training wells and testing wells. The training wells will be used to fit our neural networks, while the testing wells will be used to evaluate the performance of the trained networks on unseen data.

In [ ]:
train_well_data_path = [r".\Well Data\15_9-F-1 B.csv", r".\Well Data\15_9-F-4.csv", r".\Well Data\15_9-F-5.csv"]
test_well_data_path = [r".\Well Data\15_9-F-12.csv"]

# List of wells:
#r".\Well Data\15_9-F-1 B.csv"
#r".\Well Data\15_9-F-4.csv"
#r".\Well Data\15_9-F-5.csv"
#r".\Well Data\15_9-F-11 B.csv"
#r".\Well Data\15_9-F-12.csv"
#r".\Well Data\15_9-F-14.csv"
#r".\Well Data\15_9-F-15 C.csv"

We now use the `process_well_data()` method from the `DataProcessing` class to handle data cleaning. The data is filtered for `sandstone` intervals. We start with processing the training data.

In [ ]:
# Specify which well log features to include in the training dataset
selected_columns_train = ['BVW', 'KLOGH', 'VSH', 'GR', 'NPHI', 'RHOB', 'RT', 'LITHOLOGY'] # 'LITHOLOGY' was only included for filtering
processed_train_well_data = DataProcess.process_well_data(
    train_well_data_path,
    selected_columns_train,
    method='standard',
    train_data=True, # this indicates that this data should be used to compute the scaling parameters (mean and standard deviation for method='standard')
    show_stats=True,
    filter_lithology='Sandstone'
)

We process the testing wells next. The selected columns include the same features as in training, plus `DEPTH` for reference. As before, we filter for `sandstone` intervals and display summary statistics.

In [ ]:
selected_columns_test = ['DEPTH', 'BVW', 'KLOGH', 'VSH', 'GR', 'NPHI', 'RHOB', 'RT', 'LITHOLOGY'] # 'LITHOLOGY' was only included for filtering
processed_test_well_data = DataProcess.process_well_data(
    test_well_data_path,
    selected_columns_test,
    method='standard',
    show_stats=True,
    filter_lithology='Sandstone')

Using `scale_dataframe()`, we scale the processed training and test datasets.

In [ ]:
# scaling training dataset
scaled_train_well_data = DataProcess.scale_dataframe(processed_train_well_data, show_stats=True)

# scaling test dataset
scaled_test_well_data = [] # each well has its data stored in a separate DataFrame
for df in processed_test_well_data:
    scaled_df = DataProcess.scale_dataframe(df, show_stats=True)
    scaled_test_well_data.append(scaled_df)

Before training the neural network, it’s useful to examine how the input features relate to the target variable (`log_KLOGH`). We compute Spearman’s correlation using the scaled training data to capture monotonic relationships, and visualise them in a heatmap to identify which features are most strongly associated with permeability.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Select the input features and target from scaled_train_well_data
features = ['BVW', 'log_KLOGH', 'VSH', 'GR', 'NPHI', 'RHOB', 'log_RT']
df_corr = scaled_train_well_data[features]

# Compute Spearman correlation
corr_matrix = df_corr.corr(method='spearman')

# Plot
fig, ax = plt.subplots(figsize=(6, 5))
cax = ax.matshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
plt.colorbar(cax)

# Set labels
ax.set_xticks(range(len(features)))
ax.set_yticks(range(len(features)))
ax.set_xticklabels(features, rotation=45, ha='left')
ax.set_yticklabels(features)
ax.set_title("Spearman's Correlation Matrix (on scaled training data)", pad=15)

# Annotate each cell with correlation value
for (i, j), val in np.ndenumerate(corr_matrix.values):
    ax.text(j, i, f"{val:.2f}", ha='center', va='center', color='black')

plt.tight_layout()
plt.show()


##### III$\,\,\,\,\,\,$Neural Network Training

We now define the input and output arrays for the neural network. `X_train` holds the input feature(s), while `y_train` contains the target variable.

Both arrays are converted to NumPy arrays, and `y_train` is reshaped to match the expected dimensions for training.

In [ ]:
X_train = scaled_train_well_data[['GR', 'VSH','RHOB', 'log_RT']].to_numpy() # Input options: 'BVW', 'log_KLOGH', 'VSH', 'GR', 'NPHI', 'RHOB', 'log_RT'
y_train = scaled_train_well_data['log_KLOGH'].to_numpy().reshape(-1, 1) # Output: 'log_KLOGH'

In a similar manner, we loop through the test wells, creating `X_test` and `y_test` arrays for each.

In [ ]:
X_test_list = []
y_test_list = []

for df in scaled_test_well_data:
    X_test = df[['GR', 'VSH','RHOB', 'log_RT']].to_numpy() # Input options: 'BVW', 'log_KLOGH', 'VSH', 'GR', 'NPHI', 'RHOB', 'log_RT'
    y_test = df['log_KLOGH'].to_numpy().reshape(-1, 1) # Output: 'log_KLOGH'

    X_test_list.append(X_test)
    y_test_list.append(y_test)

We shuffle the training data to randomise the order of samples. This helps prevent the neural network from learning any unintended patterns from the original sequence of the data. The `random_state` parameter ensures the shuffle is reproducible.

In [ ]:
X_train, y_train = DataProcess.shuffle_data(X_train, y_train, random_state=42)

We create a 2D crossplot of the training data to visualise the relationship between input(s) and target. This helps assess patterns and potential correlations before training the neural network.

Given how the data was filtered earlier, the plot only shows `sandstone` intervals.

In [ ]:
visualiser.crossplot_2D(
    well_name="Training Data",
    df=scaled_train_well_data,
    x_col="log_RT", # options: 'GR', 'VSH','RHOB', 'log_RT'
    y_col="log_KLOGH",
    color_col="LITHOLOGY",
    filter_lithology="Sandstone"
)

We import the `FeedforwardNeuralNetwork` class, which provides a simple fully connected neural network implementation for regression tasks. This will be used to model the relationship between NPHI and log_KLOGH.

In [ ]:
from Lib.NeuralNetworkFunctions import FeedforwardNeuralNetwork

We initialise the feedforward neural network, as follows:
*   `input_NN` defines the number of input features (here, 1 for NPHI).
*   `hidden_NN` specifies three hidden layers, each with 48 neurons.
*   `activation_NN='relu'` applies the ReLU activation function to the hidden layers.
*   `output_NN=[1]` defines a single output neuron for predicting log_KLOGH.
*   `output_names` labels the output variable for reference.

This sets up the network architecture before training.

In [ ]:
model = FeedforwardNeuralNetwork(
    input_NN=[X_train.shape[1]],
    hidden_NN=[48, 48, 48],
    activation_NN='relu',
    output_NN=[1],
    output_names=['log_KLOGH']
)

Next, we compile the neural network. We specify the following training parameters:

*   `l_rate` sets the learning rate for the optimiser.
*   `beta_1` and `beta_2` are momentum parameters for the Adam optimiser.
*   `epsilon` is a small value to prevent division by zero.
*   `loss_type='mae'` uses mean absolute error as the loss function, which is appropriate for regression tasks.

This prepares the model for training.

In [ ]:
model.compileNNFeedforward(
    l_rate=1e-3, beta_1=0.99, beta_2=0.999, epsilon=1e-7,
    loss_type='mse' # 'l1l2', 'mae', or 'mse'
)

We train the neural network on the training data with a *validation split* of 20%. This means 20% of the training data is held out during training to monitor the model's performance on unseen samples. Training runs for 200 `epochs` with a `batch size` of 32, and the progress (including training and validation loss) is stored in `history`.

In [ ]:
history = model.train(X_train, y_train, epochs=200, batch_size=32, verbose=2,
                      validation_split=0.2)

We plot the training and validation loss over epochs to visualize how the performance of the model evolves. This helps identify underfitting, overfitting, or whether additional training might improve results.

In [ ]:
model.plot_training_loss(history)

We generate predictions for each test well using the trained neural network. For each set of test inputs, the network predicts `log_KLOGH`, and the results are stored in a list for later evaluation.

In [ ]:
predictions_test = []
for X_test in X_test_list:
    y_pred = model.NN_Feedforward.predict(X_test, verbose=0)
    predictions_test.append(y_pred)

We plot predicted `log_KLOGH` values against the actual measurements for each test well. The red dashed line represents the ideal 1:1 fit - points close to this line indicate accurate predictions, while deviations highlight areas where the model under- or over-predicts. This provides a quick visual assessment of overall model performance.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

for i, (X_test, y_test, y_pred) in enumerate(zip(X_test_list, y_test_list, predictions_test)):
    y_pred = y_pred.flatten()      # Flatten predicted array
    y_test = y_test.flatten()      # Flatten true array
    plt.figure(figsize=(6,6))
    plt.scatter(y_test, y_pred, c='blue', alpha=0.6, edgecolors='k')
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Ideal Fit')
    plt.xlabel('Actual log_KLOGH')
    plt.ylabel('Predicted log_KLOGH')
    plt.title(f'Test Well {i+1}: Predicted vs Actual')
    plt.legend()
    plt.grid(True)
    plt.show()

We can also visualise how the predicted `log_KLOGH` compares to the actual values for each test well in relation to the input feature `NPHI`. Side-by-side scatter plots allow us to see whether the model captures the trends present in the training data and highlight any deviations.

In [ ]:
# List of input features in X_test (must match the order used in training)
input_features = ['GR', 'VSH','RHOB', 'log_RT']
feature_name = 'GR'  # Feature to plot on x-axis

# Get index of the selected feature in X_test
feature_index = input_features.index(feature_name)

for i, (X_test, y_test, y_pred) in enumerate(zip(X_test_list, y_test_list, predictions_test)):
    y_pred = y_pred.flatten()
    y_test = y_test.flatten()

    # Select the feature to plot
    if X_test.ndim == 1 or X_test.shape[1] == 1:
        feature_data = X_test.flatten()  # single feature
    else:
        feature_data = X_test[:, feature_index]  # multiple features

    fig, axs = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

    # Actual values
    axs[0].scatter(feature_data, y_test, c='blue', alpha=0.6, edgecolors='k')
    axs[0].set_xlabel(feature_name)
    axs[0].set_ylabel('log_KLOGH')
    axs[0].set_title(f'Test Well {i+1} - Actual')
    axs[0].grid(True)

    # Predicted values
    axs[1].scatter(feature_data, y_pred, c='orange', alpha=0.6, edgecolors='k')
    axs[1].set_xlabel(feature_name)
    axs[1].set_title(f'Test Well {i+1} - Predicted')
    axs[1].grid(True)

    plt.tight_layout()
    plt.show()


We can also compare the neural network’s predictions of log_KLOGH with the true values along the well depth.

In [ ]:
for i, (df, y_pred) in enumerate(zip(scaled_test_well_data, predictions_test)):
    depth = df['DEPTH'].values
    y_true = df['log_KLOGH'].values
    y_pred_flat = y_pred.flatten()

    fig, axs = plt.subplots(1, 2, figsize=(6, 12), sharey=True)

    # True values
    axs[0].plot(y_true, depth, color='blue')
    axs[0].set_xlabel('log_KLOGH')
    axs[0].set_ylabel('Depth')
    axs[0].set_title(f'Test Well {i+1} - True')
    axs[0].grid(True)

    # Predicted values
    axs[1].plot(y_pred_flat, depth, color='orange')
    axs[1].set_xlabel('log_KLOGH')
    axs[1].set_title(f'Test Well {i+1} - Predicted')
    axs[1].grid(True)

    plt.gca().invert_yaxis()  # Depth increases downward
    plt.tight_layout()
    plt.show()
